In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import StratifiedKFold

import sys
sys.path.append("../src/")
from lifelong_dnn import LifeLongDNN

Using TensorFlow backend.


# Backward Transfer Efficiency

In [ ]:
n_tasks = 10
n_folds = 6

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test])
y = y[:, 0]

num_classes = len(np.unique(y))

kfold = StratifiedKFold(n_splits = n_folds)

backward_transfer_efficiencies_across_tasks_across_splits = [[] for _ in range(n_tasks)]
fold = 0
for train_fold_idxs, test_fold_idxs in kfold.split(X, y):
    print("Starting Fold: {} / {}".format(fold + 1, n_folds))
    X_train, y_train = X[train_fold_idxs], y[train_fold_idxs]
    X_test, y_test = X[test_fold_idxs], y[test_fold_idxs]
    
    X_train_across_tasks, y_train_across_tasks = [], []
    X_test_across_tasks, y_test_across_tasks = [], []
    for task_idx in range(n_tasks):
        train_idxs_of_task = np.where((y_train >= int(num_classes / n_tasks) * task_idx) & (y_train < int(num_classes / n_tasks) * (task_idx + 1)))[0]
        X_train_across_tasks.append(X_train[train_idxs_of_task])
        y_train_across_tasks.append(y_train[train_idxs_of_task] - int(num_classes / n_tasks) * task_idx)

        test_idxs_of_task = np.where((y_test >= int(num_classes / n_tasks) * task_idx) & (y_test < int(num_classes / n_tasks) * (task_idx + 1)))[0]
        X_test_across_tasks.append(X_test[test_idxs_of_task])
        y_test_across_tasks.append(y_test[test_idxs_of_task] - int(num_classes / n_tasks) * task_idx)
        
    for outer_task_idx in range(n_tasks):
        lifelong_dnn = LifeLongDNN()
        backward_generalization_errors_across_tasks = []
        for inner_task_idx in range(outer_task_idx, n_tasks):
            X_train_of_task = X_train_across_tasks[inner_task_idx]
            y_train_of_task = y_train_across_tasks[inner_task_idx]

            lifelong_dnn.add_task(X_train_of_task , y_train_of_task, epochs = 10)

            generalization_error = np.mean(y_test_across_tasks[outer_task_idx] != lifelong_dnn.predict(X_test_across_tasks[outer_task_idx], 0))
            backward_generalization_errors_across_tasks.append(generalization_error)

            print("Backward Generalization of Task {} Error Across Tasks: {}".format(outer_task_idx, backward_generalization_errors_across_tasks))

        backward_transfer_efficiencies_across_tasks = backward_generalization_errors_across_tasks[0] / np.array(backward_generalization_errors_across_tasks)
        backward_transfer_efficiencies_across_tasks_across_splits[outer_task_idx].append(backward_transfer_efficiencies_across_tasks)
    fold += 1
    
for task_idx in range(n_tasks):
    y = np.mean(backward_transfer_efficiencies_across_tasks_across_splits[task_idx], axis = 0)
    x = range(task_idx + 1, n_tasks + 1)
    plt.plot(x, y, 'o-', color = 'r')
plt.ylabel("Backward Transfer Efficiency")
plt.xlabel("Number of Tasks Seen")
plt.title("Backward Transfer Efficiencies Across Tasks")
plt.xticks(range(1, n_tasks + 1))
plt.show()

Starting Fold: 1 / 6
Backward Generalization of Task 0 Error Across Tasks: [0.574]
Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571]


Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527]
Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516]
Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516, 0.511]


Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516, 0.511, 0.499]
Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516, 0.511, 0.499, 0.483]
Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516, 0.511, 0.499, 0.483, 0.491]


Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516, 0.511, 0.499, 0.483, 0.491, 0.488]
Backward Generalization of Task 0 Error Across Tasks: [0.574, 0.571, 0.527, 0.516, 0.511, 0.499, 0.483, 0.491, 0.488, 0.474]


Backward Generalization of Task 1 Error Across Tasks: [0.619]
Backward Generalization of Task 1 Error Across Tasks: [0.619, 0.6]
Backward Generalization of Task 1 Error Across Tasks: [0.619, 0.6, 0.589]


...

Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518]
Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518, 0.494]


Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518, 0.494, 0.489]
Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518, 0.494, 0.489, 0.479]
Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518, 0.494, 0.489, 0.479, 0.471]


Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518, 0.494, 0.489, 0.479, 0.471, 0.471]
Backward Generalization of Task 2 Error Across Tasks: [0.507, 0.518, 0.494, 0.489, 0.479, 0.471, 0.471, 0.463]


Backward Generalization of Task 3 Error Across Tasks: [0.551]
Backward Generalization of Task 3 Error Across Tasks: [0.551, 0.547]
Backward Generalization of Task 3 Error Across Tasks: [0.551, 0.547, 0.558]


Backward Generalization of Task 3 Error Across Tasks: [0.551, 0.547, 0.558, 0.534]
Backward Generalization of Task 3 Error Across Tasks: [0.551, 0.547, 0.558, 0.534, 0.525]
Backward Generalization of Task 3 Error Across Tasks: [0.551, 0.547, 0.558, 0.534, 0.525, 0.524]


# Forward Transfer Efficiency

In [ ]:
n_tasks = 10
n_folds = 6

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test])
y = y[:, 0]

num_classes = len(np.unique(y))

kfold = StratifiedKFold(n_splits = n_folds)

forward_transfer_efficiencies_across_tasks_across_splits = []
for train_fold_idxs, test_fold_idxs in kfold.split(X, y):
    print("Starting Fold: {} / {}".format(len(forward_transfer_efficiencies_across_tasks_across_splits) + 1, n_folds))
    X_train, y_train = X[train_fold_idxs], y[train_fold_idxs]
    X_test, y_test = X[test_fold_idxs], y[test_fold_idxs]
    
    X_train_across_tasks, y_train_across_tasks = [], []
    X_test_across_tasks, y_test_across_tasks = [], []
    for task_idx in range(n_tasks):
        train_idxs_of_task = np.where((y_train >= int(num_classes / n_tasks) * task_idx) & (y_train < int(num_classes / n_tasks) * (task_idx + 1)))[0]
        X_train_across_tasks.append(X_train[train_idxs_of_task])
        y_train_across_tasks.append(y_train[train_idxs_of_task] - int(num_classes / n_tasks) * task_idx)

        test_idxs_of_task = np.where((y_test >= int(num_classes / n_tasks) * task_idx) & (y_test < int(num_classes / n_tasks) * (task_idx + 1)))[0]
        X_test_across_tasks.append(X_test[test_idxs_of_task])
        y_test_across_tasks.append(y_test[test_idxs_of_task] - int(num_classes / n_tasks) * task_idx)
        
    lifelong_dnn = LifeLongDNN()
    forward_transfer_efficiencies_across_tasks = []
    for task_idx in range(n_tasks):
        X_train_of_task = X_train_across_tasks[task_idx]
        y_train_of_task = y_train_across_tasks[task_idx]
        
        X_test_of_task = X_test_across_tasks[task_idx]
        y_test_of_task = y_test_across_tasks[task_idx]

        lifelong_dnn.add_task(X_train_of_task, y_train_of_task, epochs = 10)

        generalization_error_full = np.mean(y_test_of_task != lifelong_dnn.predict(X_test_of_task, task_idx))
        generalization_error_task = np.mean(y_test_of_task != lifelong_dnn.predict(X_test_of_task, task_idx, transformer_task_idxs = [task_idx]))
        forward_transfer_efficiency_of_task = generalization_error_task / generalization_error_full
        forward_transfer_efficiencies_across_tasks.append(forward_transfer_efficiency_of_task)

        print("Forward Transfer Efficiencies: {}".format(forward_transfer_efficiencies_across_tasks))
    forward_transfer_efficiencies_across_tasks_across_splits.append(forward_transfer_efficiencies_across_tasks)
    
x = range(1, n_tasks + 1)
y = np.mean(forward_transfer_efficiencies_across_tasks_across_splits, axis = 0)
plt.plot(x, y, 'o-', color = 'r')
plt.ylabel("Forward Transfer Efficiency")
plt.xlabel("Number of Tasks Seen")
plt.title("Forward Transfer Efficiencies Across Tasks")
plt.xticks(range(1, n_tasks + 1))
plt.show()